In [1]:

import pandas as pd
from experiments.musique.inference_only import macro_averaging
from knowledge_propagation.utils import io, vars, extractor
import os
import numpy as np
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
from glob import glob
from scipy.stats import describe
from thefuzz import fuzz


from knowledge_propagation.modules.evaluators import (
    ExactMatchEvaluator,
    RougeEvaluator,
    OpenAIEvaluator,
)
llm_evaluator = OpenAIEvaluator()

os.getcwd()

'/datastor1/zliu/mend/notebooks'

# Bio syn data

In [22]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/llama3.2-1B-common-date-year-after-eos-sft-bio_syn_v2-pretrain-midupper3/bio_syn_v2/base_n=100_prompt=no_w-gen_wo-icl_ice=False.xlsx")
df["is_num"] = df["is_num"].astype(float)
print("Num of rows:", len(df))
df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "is_num", "abs_diff",]].round(2)

Num of rows: 12062


KeyError: "['[A]|[Q] Acc EM', '[A]|[Q] Acc PM', 'exact_match'] not in index"

In [42]:
# macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff",]).round(2)

In [43]:
# macro_averaging(df[df["question_type"] == "efficacy"], multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff",]).round(2)
macro_averaging(df, multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", 'exact_match', "is_num"] + ["abs_diff",]).round(2)

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff
0,interval,0.04,0.68,0.04,1.0,17.75
1,minus_1,0.00,0.53,0.00,1.0,41.37
2,minus_10,0.02,0.56,0.02,1.0,45.68
3,minus_3,0.02,0.56,0.02,1.0,35.59
4,plus_1,0.02,0.56,0.02,1.0,41.71
5,plus_10,0.00,0.54,0.00,1.0,48.71
6,plus_3,0.02,0.55,0.02,1.0,31.07


In [5]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_3K_light_noshare_mid-upper3/bio_syn_v2/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_all_propagation_ood-question.xlsx")
# /u/zliu/datastor1/mend/debug_exp_output/Llama-3.2-1B-common-date-year-after-eos-sft-bio_syn_v2-pretrain-all_clm-baseline_lr=1e-05_epoch=4.0/all_results_ood.xlsx
# 
df["is_num"] = df["is_num"].astype(float)
print("Num of rows:", len(df))
df.describe()[["is_num", "abs_diff",]].round(2)

def h(row):
    if "_year" in row["question_tag"]:
        return "year_facts"
    elif "_place" in row["question_tag"]:
        return "country_facts"
    elif "plus_" in row["question_tag"] :
        return "arithmetics(+)"
    else:
        assert "minus_" in row["question_tag"]
        return "arithmetics(-)"
df["question_type"] = df.apply(h, axis=1)

Num of rows: 15606


In [6]:
df["EM"] = df["abs_diff"] == 0
df["EM"] = df["EM"].astype(float)

In [7]:
df[df["question_key"] == "unaliased_question"].describe()

,id,predicted_answer_idx,is_num,abs_diff,diff,contain_ans,__index_level_0__,llm_accuracy,EM
count,7803.000000,7803.0,7803.000000,4200.000000,4200.000000,7803.000000,3603.000000,3603.000000,7803.000000
mean,49.463540,0.0,0.538255,7.707143,3.149048,0.102268,7855.903414,0.237025,0.102268
std,28.863414,0.0,0.498566,7.647942,10.391648,0.303020,4504.597657,0.294084,0.303020
min,0.000000,0.0,0.000000,0.000000,-50.000000,0.000000,121.000000,0.100000,0.000000
25%,24.500000,0.0,0.000000,2.000000,-2.000000,0.000000,3962.000000,0.100000,0.000000
50%,49.000000,0.0,1.000000,6.000000,1.000000,0.000000,7815.000000,0.100000,0.000000
75%,74.000000,0.0,1.000000,12.000000,10.000000,0.000000,11704.500000,0.100000,0.000000
max,99.000000,0.0,1.000000,80.000000,80.000000,1.000000,15605.000000,1.000000,1.000000


In [229]:
# macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=[ "is_num", "abs_diff", "llm_accuracy"]).round(2)
# macro_averaging(df[df["question_key"] == "question"], multi_level_averaging=["question_type", "id"], metrics=["is_num", "abs_diff", "llm_accuracy"]).round(2)
macro_averaging(df[df["question_key"] == "unaliased_question"], multi_level_averaging=["question_type", "id"], metrics=[ "is_num", "abs_diff", "llm_accuracy"]).round(2)

,question_type,is_num,abs_diff,llm_accuracy
0,arithmetics(+),1.0,2.65,NaN
1,arithmetics(-),1.0,2.13,NaN
2,country_facts,0.0,NaN,0.27
3,year_facts,0.0,NaN,0.19


In [210]:
# set(df[(df["question_key"] == "question") & (df["question_type"] == "country_facts") & (df["llm_accuracy"] > 0.1)]["predicted_answer"].to_list())

In [9]:
3853*2

7706

In [107]:
# macro_averaging(df[df["question_key"] == "question"], multi_level_averaging=["question_tag", "id"], metrics=[ "is_num", "abs_diff", "llm_accuracy"]).round(2)
macro_averaging(df[df["question_key"] == "unaliased_question"], multi_level_averaging=["question_tag", "id"], metrics=[ "is_num", "abs_diff", "llm_accuracy"]).round(2)

,question_tag,is_num,abs_diff,llm_accuracy
0,efficacy_common_fact_birth_place,0.0,NaN,0.27
1,efficacy_common_fact_birth_year,0.0,NaN,0.63
2,efficacy_common_fact_career_year,0.0,NaN,0.57
3,efficacy_common_fact_death_year,0.0,NaN,0.56
4,efficacy_minus_11,1.0,3.76,0.52
5,efficacy_minus_13,1.0,5.49,0.60
6,efficacy_minus_14,1.0,6.29,0.60
7,efficacy_minus_15,1.0,7.90,0.44
8,efficacy_minus_16,1.0,7.58,0.45
9,efficacy_minus_4,1.0,4.49,0.49


# Country results

In [ ]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-country_3K_light_noshare_top3/country_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_all_propagation_ood-question.xlsx")
print("Num of rows:", len(df))
df["predicted_answer"] = df["predicted_answer"].astype(str)

# df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy",]].round(2)

Num of rows: 7706


In [ ]:
# macro_averaging(df, multi_level_averaging=["stage", "input"], metrics=metrics) * 100
macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", '[Q][A] Acc EM', "[Q][A] Acc PM"] + ["exact_match", "llm_accuracy"]).round(2)

KeyError: "[('[A]|[Q] Acc EM', 'mean'), ('[A]|[Q] Acc PM', 'mean'), ('[Q][A] Acc EM', 'mean'), ('[Q][A] Acc PM', 'mean'), ('exact_match', 'mean')] not in index"

In [22]:
macro_averaging(df[df["question_type"] == "efficacy"], multi_level_averaging=["question_tag", "id"], metrics=["exact_match", "llm_accuracy"]).round(2)
# macro_averaging(df[df["question_type"] == "ood_efficacy"], multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy"]).round(2)

,question_tag,exact_match,llm_accuracy
0,efficacy_compare-city,0.80,0.82
1,efficacy_compare-continent,0.70,0.73
2,efficacy_compare-country,0.81,0.83
3,efficacy_continent,0.58,0.62
4,efficacy_country,0.43,0.49


In [ ]:
df = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/llama3.2-1B-common-date-year-after-eos-sft/bio_syn_v2_ood/base_n=100_prompt=no_w-gen_wo-icl_ice=False.xlsx")
print("Num of rows:", len(df))
df["predicted_answer"] = df["predicted_answer"].astype(str)

Num of rows: 15606


In [186]:
# len(df[df["question_key"] == "question"])
df[df["question_key"] == "question"]["question_tag"].value_counts().to_dict()

{'common_fact_birth_place': 1289,
 'common_fact_career_year': 785,
 'common_fact_death_year': 785,
 'common_fact_birth_year': 744,
 'minus_11': 300,
 'minus_13': 300,
 'minus_14': 300,
 'minus_15': 300,
 'minus_16': 300,
 'minus_4': 300,
 'minus_5': 300,
 'plus_11': 300,
 'plus_13': 300,
 'plus_14': 300,
 'plus_15': 300,
 'plus_16': 300,
 'plus_4': 300,
 'plus_5': 300}

In [181]:
# macro_averaging(df, multi_level_averaging=["question_type", "id"], metrics=[ "is_num", "abs_diff", "llm_accuracy"]).round(2)
# macro_averaging(df[df["question_key"] == "question"], multi_level_averaging=["question_tag", "id"], metrics=["llm_accuracy"]).round(2)
macro_averaging(df[df["question_key"] == "unaliased_question"], multi_level_averaging=["question_tag", "id"], metrics=[ "llm_accuracy"]).round(2)

,question_tag,llm_accuracy
0,efficacy_common_fact_birth_place,0.28
1,efficacy_common_fact_career_place,0.28
2,efficacy_common_fact_death_place,0.28


In [26]:
valid_df_content = []
for i in range(len(df)):
    r = df.iloc[i]
    if any(x in r["predicted_answer"].lower() for x in ["yes", "no"]):
        valid_df_content.append(r)
valid_df = pd.DataFrame(valid_df_content)
len(valid_df)

61

In [126]:
macro_averaging(valid_df, multi_level_averaging=["question_tag", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy"]).round(2)

,question_tag,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,llm_accuracy
0,ood_efficacy_border,0.00,0.50,0.00,0.52
1,ood_efficacy_non_border,0.14,0.57,0.14,0.23


In [101]:
valid_df.describe()[["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", "exact_match", "llm_accuracy",]].round(2)

,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,llm_accuracy
count,31.0,31.00,31.0,31.00
mean,0.0,0.48,0.0,0.65
std,0.0,0.09,0.0,0.43
min,0.0,0.00,0.0,0.10
25%,0.0,0.50,0.0,0.10
50%,0.0,0.50,0.0,1.00
75%,0.0,0.50,0.0,1.00
max,0.0,0.50,0.0,1.00


In [184]:
for q_tag, q_df in df[df["stage"] == "pre-edit"].groupby("question_tag"):
    print(q_tag + ":", sum(isinstance(x, str) for x in q_df["predicted_answer"].to_list()))

multi_hop_efficacy_q0: 999
single_hop_efficacy_q0: 802
single_hop_efficacy_q1: 744


In [45]:
dev_ids = [ex["id"] for ex in io.load_jsonlines(f"{vars.DATA_DIR}/musique_mend/2hop_musique_ans_v1.0_dev.jsonl")[:1000]]

In [225]:
save_dir = "/u/zliu/datastor1/mend/debug_exp_output/Llama-3.2-1B-common-date-year-after-eos-sft-bio_syn_v2-pretrain-all_clm-baseline_lr=1e-05_epoch=4.0/individual_results_bio_syn_data_v2_ood_text"
all_df = pd.concat([pd.read_excel(f) for f in glob(f"{save_dir}/*.xlsx", recursive=True)])
len(all_df)

15606

In [226]:
all_df.to_excel(f"{save_dir}/../all_results_ood.xlsx", index=False)

In [6]:
all_df["is_num"] = all_df["is_num"].astype(float)

In [7]:
all_df.describe()

,id,answer,predicted_answer_idx,exact_match,is_num,abs_diff,diff,[A]|[Q] Acc EM,[A]|[Q] Acc PM,[Q][A] Acc EM,[Q][A] Acc PM
count,100.000000,100.000000,100.0,100.000000,100.000000,73.000000,73.000000,100.000000,100.000000,100.0,100.000000
mean,49.500000,1957.310000,0.0,0.310000,0.730000,13.493151,8.041096,0.300000,0.790000,0.0,0.438087
std,29.011492,30.458017,0.0,0.464823,0.446196,15.383183,18.859450,0.460566,0.161746,0.0,0.068536
min,0.000000,1904.000000,0.0,0.000000,0.000000,0.000000,-48.000000,0.000000,0.500000,0.0,0.285714
25%,24.750000,1932.750000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.750000,0.0,0.418731
50%,49.500000,1954.500000,0.0,0.000000,1.000000,10.000000,0.000000,0.000000,0.750000,0.0,0.428571
75%,74.250000,1977.500000,0.0,1.000000,1.000000,20.000000,19.000000,1.000000,1.000000,0.0,0.500000
max,99.000000,2022.000000,0.0,1.000000,1.000000,61.000000,61.000000,1.000000,1.000000,0.0,0.611111


In [20]:
(macro_averaging(all_df, multi_level_averaging=["question_type", "id"], metrics=["[A]|[Q] Acc EM", "[A]|[Q] Acc PM", ] + ["exact_match", "is_num", "abs_diff", "diff"])).round(2) # llm_accuracy '[Q][A] Acc EM', "[Q][A] Acc PM"

,question_type,[A]|[Q] Acc EM,[A]|[Q] Acc PM,exact_match,is_num,abs_diff,diff
0,efficacy,0.00,0.58,0.00,0.77,13.68,9.57
1,specificity,0.23,0.76,0.23,1.00,6.99,1.22


In [25]:
efficacy_df = all_df[all_df["question_type"] == "efficacy"]

In [30]:
nan_efficacy_df = efficacy_df[efficacy_df["is_num"] == 0]

In [12]:
# for i, row in nan_efficacy_df.iterrows():
#     print(row["question"])
#     print(row["predicted_answer"])
#     print()

In [13]:
# born_question = 0
# for i, row in efficacy_df.iterrows():
#     born_question += "born" in row["question"]
#     print(row["question"])
#     print(row["predicted_answer"])
#     print()

In [24]:
mend_1k_n_question_efficacy = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n-question.xlsx")

In [25]:
mend_1k_n_1_question_efficacy = pd.read_excel("/u/zliu/datastor1/mend/debug_exp_output/common-date-year-after_1K/bio_syn/mend_eval_loss=clm_input=seen_n=100_prompt=no_w-gen_wo-icl_e_n+1-question.xlsx")

In [26]:
len(mend_1k_n_question_efficacy[mend_1k_n_question_efficacy["exact_match"] == 1])

14

In [36]:
mend_1k_n_question_correct_ids = set((mend_1k_n_question_efficacy[mend_1k_n_question_efficacy["exact_match"] == 1]["id"].to_list()))

In [35]:
mend_1k_n_1_question_correct_ids = set(mend_1k_n_1_question_efficacy[mend_1k_n_1_question_efficacy["exact_match"] == 1]["id"].to_list())

In [37]:
mend_1k_n_question_correct_ids

{20, 28, 35, 38, 41, 46, 48, 50, 54, 56, 76, 82, 84, 89}

In [39]:
mend_1k_n_1_question_correct_ids.intersection(mend_1k_n_question_correct_ids)

{89}

In [40]:
len(mend_1k_n_1_question_correct_ids)

62